<img src='https://github.com/LinkedEarth/Logos/raw/master/PYLEOCLIM_logo_HORZ-01.png' width="800">

# 5. Fishing for Correlations at Sea
Correlation analysis, despite its simplicity and many shortcomings, remains a centerpiece of empirical analysis in many fields, particularly the paleosciences. Computing correlations is trivial enough; the difficulty lies in properly assessing their significance. Of particular importance are four considerations:

1. __Persistence__, which violates the standard assumption that the data are independent (which underlies the classical test of significance implemented, e.g. in Excel).
1. __Time irregularities__, for instance comparing two records with different time axes, possibly unevenly spaced (which standard software cannot deal with out of the box)
1. __Age uncertainties__, for example comparing two records, each with an ensemble of plausible chronologies (generated, for instance, by a Bayesian age model)
1. __Test multiplicity__ aka the "Look Elsewhere effect", which states that repeatedly performing the same test can result in unacceptably high type I error (accepting correlations as significant, when in fact they are not). This arises e.g. when correlating a paleoclimate record with an instrumental field, assessing significance at thounsands of grid points at once, or assessing significance within an age ensemble.

Accordingly,  Pyleoclim facilitates an assessment of correlations that deals with all these cases, makes the necessary data transformations transparent to the user, and allows for one-line plot commands to visualize the results. We start by loading a few useful packages:

In [ ]:
import matplotlib.pyplot as plt    
import pyleoclim as pyleo
pyleo.set_style('web')  # set the visual style
import numpy as np
import xarray as xr

## A case study from Crystal Cave
In this notebook we reproduce the case of [Hu et al, 2017](http://dx.doi.org/10.1016/j.epsl.2016.11.048), particularly the example of their section 4, which illustrates several of these pitfalls at once. The example illuminates the issue of relying too strongly on correlations between a paleoclimate record and an instrumental field to interpret the record. Before we start, a disclaimer: the studies investigated in this paper are by no means isolated cases. They just happened to be cases that we knew about, and thought deserved a second look in light of more rigorous statistics. The same study could have been written by subsituting any number of other records interpreted, wholly or in part, on the basis of correlations. Accordingly, what follows should not be viewed as an indictment of a particularly study or group of authors, rather, at how easy it is by the best-intentioned scientists to get fooled by spurious correlations, and (thankfully), how easy we've made it not get similarly fooled by carrying out this analysis with `pyleoclim`. 


### The Crystal Cave record

The example uses the speleothem record of [McCabe-Glynn et al , 2013](https://www.nature.com/articles/ngeo1862) from Crystal Cave, California, in the Sequoia National Forest.  Of interest to us is the $\delta^{18}O$ record, which the authors interepret as reflecting sea-surface temperatures (SST) in the Kuroshio Extension region of the West Pacific. This is a strong claim, given that no mechanistic link is proposed,  and relies entirely on an analysis of correlations between the record and instrumental SST.  

We first load and plot this record:

In [ ]:
d = pyleo.Lipd('../data/Crystal.McCabe-Glynn.2013.lpd')
cc = d.to_LipdSeries(2)   

Let's do a quick plot to check that we have what we want:

In [ ]:
fig, ax = cc.plot(mute=True)
pyleo.showfig(fig, close=True)

Notice how the code harvested the correct metadata from the LiPD file. If everything is in its right place, it makes it easy to exploit that information. If you're feeling more frisky, you can even ask for a whole dashboard, including a spectrum:

In [ ]:
cc.dashboard(metadata=False)

This is a very high resolution record, with near-annual spacing (check it), and a broadly red spectrum that exhibits a number of spectral peaks at interannual and decadal scales.


### SST data

The original paper correlated the above record against the Kaplan SST dataset.  In this notebook we instead use the [HadSST4 dataset](https://www.metoffice.gov.uk/hadobs/hadsst4/index.html),  which is more up to date, and which we first download via `wget`. (~8Mb)

In [ ]:
!wget https://www.metoffice.gov.uk/hadobs/hadsst4/data/netcdf/HadSST.4.0.1.0_median.nc
!mv HadSST.4.0.1.0_median.nc ../data

Next we load it via the excellent `xarray` package.

In [ ]:
ds = xr.open_dataset('../data/HadSST.4.0.1.0_median.nc')
print(ds)

As an example, let's only consider the Northern Hemisphere Pacific Ocean. For practicality, let's first adjust the coordinate system so that longitude is expressed from 0 to 360 degrees instead of 180W to 180E:

In [ ]:
ds_rolled = ds.assign_coords(longitude=(ds.longitude % 360)).roll(longitude=(ds.dims['longitude'] // 2))
ds_rolled

Now let's select the needed data:

In [ ]:
ds_sel = ds_rolled.sel(longitude=slice(120,280),latitude=slice(0,90))
ds_sel

## Pitfall #1: Persistence

Persistence is the tendency of many geophysical timeseries (particularly in paleoclimatology) to show some kind of memory: consecutive observations tend to resemble each other, resulting in timeseries that have fairly broad trends and low-frequency fluctuations, and comparatively little high-frequency fluctuations. 

This has an important consequence: the standard assumption of independence, which undergirds much of frequentist statistics, is violated in this case. In a timeseries with $n$ fully independent observations (e.g. white noise), the degrees of freedom for the variance are $DOF = n -1$  But if memory is present, this number can be drastically reduced. 

### Single location
Let us look at a random location and build some intuition. First, we need to compute montly anomalies and annualize them. `xarray` makes that easy (4 lines of code), so let's look at the result:

In [ ]:
st = ds['tos'].sel(latitude=32.5, longitude = 142.5, method='nearest')  # 32.5N 142.5W near Kuroshio Extension
climatology = st.groupby("time.month").mean("time")
anomalies = st.groupby("time.month") - climatology
st_annual = anomalies.groupby("time.year").mean("time")
f, ax = plt.subplots()
st_annual.plot()
pyleo.showfig(f)

Notice the coverage gaps in the 1860s. This, and the fact that the Crystal Cave chronology is not uniformly spaced, would ordinarily make it challenging (at the vary least, bothersome) to compare the two series, requiring some form of interpolation or binning. Pyleoclim does all this for you under the hood.

To enjoy these benefits, however, let us turn the temperature data into a _Series_ object.

In [ ]:
stts = pyleo.Series(time=st_annual.coords['year'].values,
                    time_unit ='year CE', 
                    value=st_annual.values,
                    value_unit = 'C')

Now we can compute correlations with the Crystal Cave record. 

In [ ]:
corr_res = stts.correlation(cc)
print(corr_res.r)

Quite a few things happened here. First, `pyleoclim` was smart enough to figure out a common timespan between the two records, and used linear interpolation to align the two timeseries on a common axis. 

The resulting correlation is $r=0.38$. Is it significant?

The standard way to assess this (embedded in countless computing packages), is with a t-test using the test statistic: $$T = \frac{r \sqrt{n-2}}{\sqrt{1-r^2}}$$

If we plug the numbers in, we get the following:  

In [ ]:
ccs = cc.slice([1854,2020])
n = len(ccs.time)
nu = n-2
r = corr_res.r
T  = r *np.sqrt(nu)/(np.sqrt(1-r**2))
print("The test statistic is "+ str(T))

Under standard assumptions (the data are independent and identically distributed), $T$ follows Student's $t$ distribution (the first [Guiness-soaked distribution](https://priceonomics.com/the-guinness-brewer-who-revolutionized-statistics/) in history). If we make the same assumption and use the $t$ distribution conveniently programmed for us by SciPy, we can compute the p-value (the probability to observe a test statistic at least as large as this one, under this distribution) thusly:

In [ ]:
from scipy.stats import t
pval = 1-t.cdf(T,nu)
print("The p-value is "+ str(pval))

In other words, using the classic test for the significance of correlations "out of the box", one would conclude that SST at 42N, 164E shares so much similarity with the Crystal Cave record that there are only a few chances in 10,000 that this could have happened randomly. In other words, it looks _rather_ significant. 

But let's take a step back. That test (which is the one that most computing packages, including Excel, will do for you out of the box), is completely inappropriate here. Why? Because it tramples over the concept of persistence with gleeful impunity. That is, it assumes that consecutive observations bear no resemblance to each other, which is true neither of the Crystal Cave nor the instrumental target. That is to say: because temperature in one year tends to resemble temperature in the previous or following year (same for $\delta^{18}O$, the data are anything but independent. 

Going back to the result of the `correlation()` command, let's look at its full output:

In [ ]:
print(corr_res)

Notice that the p-value has been estimated to be 15% (`'p': 0.15`), and therefore the correlation is not deemed significant (`'signif': False`) at the 5% level (`'alpha': 0.05`).

How did `pyleoclim` arrive at such a different conclusion? By not applying irrelevant assumptions, of course! To know what method was applied exactly precisely, consult the [documentation](https://pyleoclim-util.readthedocs.io/en/latest/index.html) or the docstring. 

**Exercise 5.1** What method does `correlation` use by default to assess significance and what are its assumptions? (hint: check out "correlation" on [Read The Docs](https://pyleoclim-util.readthedocs.io/en/latest/index.html), under "The Pyleoclim user interface"), or type `stts.correlation?` at the prompt.

**Answer 5.1** YOUR WORDS HERE



**Exercise 5.2** There are in fact 3 ways to make this determination in Pyleoclim. Try the other two in the cells below, and compare their p-values. Do they give consistent answers or not?

In [ ]:
## your code here ##

## Pitfall #2: Multiple testing

The foregoing shows how to properly assess significance at just one location. How would we go about conducting a similar test for an entire field? Let us first try the naive approach: recursively carry out the same test as above at all grid points.  For this, we need not only to loop over grid points, but also store the p-values for later analysis. To save time, we'll use the `ttest` option for `correlation()`, knowing that it is rather approximate. Also, we need to exclude points that have too few observations.  The loop below achieves that. 

In [ ]:
nlon = len(ds_sel['longitude'])
nlat = len(ds_sel['latitude']) 
pval = np.empty((nlon,nlat)) # declare array to store pvalues
corr = np.empty_like(pval) # declare empty array of identical shape
alpha = 0.05
slon, slat = [], [];
for ji in range(nlon):
    for jj in range(nlat):     # TODO add progress bar 
        st = ds_sel['tos'][:,jj,ji]
        climatology = st.groupby("time.month").mean("time")
        anomalies = st.groupby("time.month") - climatology
        st_annual = anomalies.groupby("time.year").mean("time")
        #  test if at least 100 non-NaNs
        noNaNs = len(np.where(np.isnan(st_annual) == False)[0]) # number of valid years
        sstvar = st_annual.var()
        if noNaNs >= 100 and sstvar >= 0.01:
            print("Computing correlation at " + str(ds_sel.latitude[jj].values) + 'N, ' + str(ds_sel.longitude[ji].values) + 'E')
            sttb = pyleo.Series(time=st_annual.coords['year'].values,
                        time_unit ='year CE', 
                        value=st_annual.values,
                        value_unit = 'C')
            corr_res = sttb.correlation(cc, settings={'method':'ttest'})
            pval[ji,jj] = corr_res.p
            corr[ji,jj] = corr_res.r
            if pval[ji,jj] < alpha:
                slon.append(ds_sel.longitude[ji])
                slat.append(ds_sel.latitude[jj])
        else:  
            pval[ji,jj] = np.nan; corr[ji,jj] = np.nan

In [ ]:
pvals = pval.flatten() # make the p-value array a 1D one
pvec = pvals[pvals<1] # restrict to valid probabilities as there are a few weird values.
nt = len(pvec)
print(nt) # check on the final number

We found 327 with enough data for a meaningful comparison, and 23 locations that pass the test. Where are they? To gain insight, let us plot the correlations and indicate (by shading) which are deemed significant:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import seaborn as sns

land_color=sns.xkcd_rgb['light grey']
ocean_color=sns.xkcd_rgb['light grey']

fig = plt.figure(figsize=[8, 6])
ax = plt.subplot(projection=ccrs.PlateCarree(central_longitude=180))

# map
ax.add_feature(cfeature.LAND, facecolor=land_color, edgecolor=land_color)
ax.add_feature(cfeature.OCEAN, facecolor=ocean_color, edgecolor=ocean_color)
ax.coastlines()

transform = ccrs.PlateCarree()
latlon_range = (120, 280, 0, 70)
lon_min, lon_max, lat_min, lat_max = latlon_range
lon_ticks = [60, 120, 180, 240, 300]
lat_ticks = [-90, -45, 0, 45, 90]

ax.set_extent(latlon_range, crs=transform)
lon_formatter = LongitudeFormatter(zero_direction_label=False)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
lon_ticks = np.array(lon_ticks)
lat_ticks = np.array(lat_ticks)
mask_lon = (lon_ticks >= lon_min) & (lon_ticks <= lon_max)
mask_lat = (lat_ticks >= lat_min) & (lat_ticks <= lat_max)
ax.set_xticks(lon_ticks[mask_lon], crs=transform)
ax.set_yticks(lat_ticks[mask_lat], crs=transform)

# contour
clevs = np.linspace(-0.9, 0.9, 19)
#corr_r, lon_r = rotate_lon(corr.T, lon)  # rotate the field to make longitude in increasing order and convert to range (0, 360)
im = ax.contourf(ds_sel.longitude, ds_sel.latitude, corr.T, clevs, transform=transform, cmap='RdBu_r', extend='both')

# significant points
plt.scatter(x=slon, y=slat, color="purple", s=3,
            alpha=1,
            transform=transform) 


# colorbar
cbar_pad = 0.05
cbar_orientation = 'vertical'
cbar_aspect = 10
cbar_fraction = 0.15
cbar_shrink = 0.5
cbar_title = 'R'
cbar = fig.colorbar(im, ax=ax, orientation=cbar_orientation, pad=cbar_pad, aspect=cbar_aspect,
                    fraction=cbar_fraction, shrink=cbar_shrink)
cbar.ax.set_title(cbar_title)

pyleo.showfig(fig)

The purple dots on the map are the locations of the gridpoints where the p-values fall under 5%, and they naturally correspond to the regions of highest correlations, though (and this is suspicious) they are rather randomly scattered across the domain.  
We might be tempted to declare victory and hail them as "significant", but not so fast! 
Our correlation test, nifty though it is, isn't infallible. In fact, conducting tests at the 5% level (equivalently, the 95% confidence level) specifically means that we expect 5% of our tests to return spurious results, just from chance alone. We just carried out $n_t$ tests, so we expect $0.05*n_t \approx 16 $ of those results to be bunk, right out of the gate.  Which ones can we trust?

Let us rank order the p-values of all 327 tests and plot them as in Hu et al (2017), Fig 2.

In [ ]:
#check i/m vs. p-values
indexm = np.arange(1,nt+1,1)
im = 1.0*indexm / nt
thres = 0.05*im
pvec_s = sorted(pvec)
smaller=[]
small_index=[]
larger=[]
large_index=[]

n=0
for pp in pvec_s:
    if pp <=0.05:
        smaller.append(pp)
        small_index.append(im[n])
    else:
        larger.append(pp)
        large_index.append(im[n])
    n=n+1

f, ax = plt.subplots()
#plt.plot(im,pvec_s,'kx',markersize=1.5,label='p-values',alpha=0.3)
plt.plot(im,thres,label='FDR threshold')
plt.plot(small_index,smaller,'ro',markersize=1.5,label='p-values below threshold')
plt.plot(large_index,larger,'ko',markersize=1.5,label='p-values above threshold',alpha=0.3)
plt.axhline(y=0.05,linestyle='dashed',color='silver',label='5% threshold')
plt.xlabel(r'$i/n_t$',fontsize=14)
plt.ylabel(r'$p_i$',fontsize=14)
plt.title('Correlation p-values')
plt.legend()
#plt.tick_params(labelsize=14)
pyleo.showfig(f)

One solution to this is the False Discovery Rate (aka **FDR**), which was devised in a celebrated 1995 paper [(Benjamini & Hochberg, 1995)](https://rss.onlinelibrary.wiley.com/doi/abs/10.1111/j.2517-6161.1995.tb02031.x). The idea is to look not just for the p-values under 5% (red dots in the figure above), but for the fraction of those under the blue line, which guards against the false discoveries one expects out of repeatedly testing the same hypothesis over and over again. Luckily for you, we have this coded up in `pyleoclim` (you knew we would). One way to access it is thus:

In [ ]:
fdr_res = pyleo.utils.correlation.fdr(pvec_s)
print(fdr_res)

And with this treatment, exactly 0 gridpoints pass the test. 

**Exercise 5.3** redo this with other correlation methods and see if the results change. A little patience is required for the other two methods, which loop over surrogates. We suggest using a moderate number of simulations (`nsim` < 500) to do so; please check the documentation of `corr_sig` for how to reduce this number, which is 1000 by default. 


## Pitfall #3: Age uncertainties

As if the first two pitfalls weren't enough, there is a third difficulty in this comparison: beautiful though U/Th chronologies may be, they are not perfect (do not believe people who tell you otherwise!). Such chronological uncertainties must be taken into account as well. 

It turns out that the LiPD file loaded above contains not only the raw U/Th dates (dig for them if you're curious), but also an ensemble of age-depth relationships derived from those dates, via the Bayesian age model [BChron](https://cran.r-project.org/web/packages/Bchron/vignettes/Bchron.html). Let us load those 1000 draws from the posterior distribution of ages and put them in a place where `pyleoclim` will be able to work with them:

In [ ]:
cc_ens = cc.chronEnsembleToPaleo(d)

There are two ways to plot this ensemble. First, as a series of traces:

In [ ]:
fig, ax = cc_ens.plot_traces(mute=True)
cc.plot(color='black', ax = ax)
pyleo.showfig(fig)

It is quite plain that any of the record's main swings can be swung back and forth by up to decades. Another way to see this is to plot various quantiles as an envelope:

In [ ]:
fig, ax = cc_ens.common_time(method='interp').plot_envelope(ylabel = cc.value_name, mute=True)
cc.plot(color='black', ax = ax, linewidth=1)
pyleo.showfig(fig)

Notice how the quantiles are computed over only part of the interval covered by the original series; that is because some age models end up being more compressed or stretched out than the original, and we need a common interval to compute quantiles. In particular, notice how the base of the record could really be anywhere between 850 and 1100 AD.

Now, what we'd like to do is repeat the exercise of Part 1, correlating the same SST timeseries from 32.5N 142.5W (near the Kuroshio Extension), not just with the published chronology, but this whole ensemble. Remember how we had computed things:

In [ ]:
corr_res = stts.correlation(cc)
print(corr_res.r)

Though of course, we could have done just the reverse, as correlation is a symmetric operator:

In [ ]:
corr_res = cc.correlation(stts)
print(corr_res.r)

Now, consider the task ahead: you must iterate over all ensemble members, taking care of aligning their time axis to that of HadSST4; you must compute the correlation and establish its significance with a sensible test (say, Ebisuzaki's isospectral test, see Pitfall #1), you need watch out for test multiplicity (see Pitfall #2) _and_  you need to visualize the results in an intuitive way. Don't you wish this has all been conveniently coded for you?



![Your wish](https://am23.mediaite.com/tms/cnt/uploads/2020/12/Life-Is-Good-but-It-Can-Be-Better-With-These-Max-Lord-Memes.jpg)


YOUR WISH HAS BEEN GRANTED!!!

To keep computing time manageable, let's reduce the number of isospectral simulations to 500, and correlate the ensemble and the SST series:

In [ ]:
nsim = 500
corr_Kuroshio = cc_ens.correlation(stts,settings={'nsim':nsim}) 

Now here's the best part: the `corr_Kuroshio` is an object that contains everything you want: the vector of correlations, the p-values, _and_ a method to plot them all:

In [ ]:
corr_Kuroshio.plot()

Several things jump out at once:
1. the correlation histogram is relatively symmetric, meaning that age uncertainties are able to completely overturn the original correlation of about 0.31 : nearly half of the ensemble exhibits negative correlations with SST).  0.31 (9% of common variance) was exactly a contender for winning the correlation olympics, but it was at least positive.
1. As shown in green, only a few \% of the 500*100 correlations just computed are judged significant (note: this would change somewhat every time you run the calculation, based on the randomness of the phase randomization procedure. If reproducibility is essential, you  may also input a [random seed](https://en.wikipedia.org/wiki/Random_seed) to ensure that the same exact random sequence is used from iteration to iteration. 
1. Once we take test multiplicity into account via the False Discovery Rate (orange bar), only a fraction of a \%  is deemed significant. 

**Exercise 5.4** redo this at other locations and see if the results change. you may consider picking a patch of the North Pacific and applying this test recursively at each location. 

In [ ]:
## your code here ##

## Conclusions


**Exercise 5.5**  To bring it all together, summarize everything that can go wrong when fishing for correlations (at sea, or on land). Can you think of other papers where correlations with paleoclimate records might have been used unwisely? (no names needed... Just keep that in mind in your own work and point people to more defensible ways of dealing with this danger, i.e. Pyleoclim!) 

**Answer 5.5** YOUR WORDS HERE